In [ ]:
!pip install yfinance

In [ ]:
!pip install autots

In [ ]:
from autots import AutoTS
import joblib
from sklearn.metrics import mean_absolute_error, mean_squared_error
import pandas as pd
import yfinance as yf

In [ ]:
# Loading the data
data = pd.read_csv('/content/df_Prophet_Autots.csv', parse_dates=['Time Serie'], index_col='Time Serie')
data.head()

In [ ]:
# 2. Split into train/test (last 60 observations as test set)
test_data = data.iloc[-60:]  # Last 60 rows
train_data = data.iloc[:-60]  # All rows except last 60

In [ ]:
model = AutoTS(
    forecast_length=60,
    frequency='infer',
    ensemble='simple',
    max_generations=5,
    num_validations=3
)
model = model.fit(train_data)

In [ ]:
# 4. Generate forecasts
prediction = model.predict()
forecast = prediction.forecast

In [ ]:
forecast.index = test_data.index

In [ ]:
# 5. Compare forecast with actual test data
print("Forecasted values:")
forecast.head()


In [ ]:
print("\nActual test values:")
test_data.head()

In [ ]:
# Initialize model with 60-day forecast
model = AutoTS(
    forecast_length=60,
    frequency='infer',  # Automatically detects time frequency
    ensemble='simple',
    max_generations=5,  # Increase for better accuracy (slower)
    num_validations=3   # Increase for more robust validation
)

# Fit model directly on DataFrame (date index + currency columns)
model = model.fit(data)

# Generate forecasts
prediction = model.predict()
forecast = prediction.forecast

In [ ]:
# Compare forecast with actual test data
print("Forecasted values:")
print(forecast.head())
print("\nActual test values:")
print(test_data.head())

In [ ]:
forecast.to_csv('forecast_results.csv', index=True)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
for column in forecast.columns:
    plt.figure(figsize=(10, 6))
    plt.plot(forecast.index, forecast[column], label='Forecast', color='blue')
    plt.plot(test_data.index, test_data[column], label='Actual', linestyle='--', color='orange')
    plt.title(f'{column} Exchange Rate: Forecast vs Actual')
    plt.xlabel('Date')
    plt.ylabel('Rate')
    plt.legend()
    plt.grid(True)
    plt.show()

#Save forecast to CSV
forecast.to_csv('forecast_results.csv', index=True)

In [ ]:
import matplotlib.pyplot as plt

# Plot forecasts for each currency on separate plots
for column in forecast.columns:

    plt.figure(figsize=(10, 6))
    # Plot predicted values
    plt.plot(forecast.index, forecast[currency], label=f'Forecast: {column}', color='blue')

  # Plot actual values
    if 'actual_df' in locals() and currency in actual_df.columns:
        plt.plot(actual_df.index, actual_df[currency], label=f'Actual {currency}', linestyle='dashed', color='orange')

    plt.title(f'Predicted vs Actual Exchange Rates for {currency}')
    plt.xlabel('Date')
    plt.ylabel('Exchnage Rate')
    plt.legend()
    plt.grid()
    plt.show()


In [ ]:
# Calculate metrics for each column (currency)
for column in forecast.columns:
    y_true = test_data[column].values
    y_pred = forecast[column].values

    # MAE
    mae = mean_absolute_error(y_true, y_pred)
    # RMSE
    rmse = mean_squared_error(y_true, y_pred, squared=False)
    # MAPE
    mape = (abs((y_true - y_pred) / y_true)).mean() * 100

    print(f"{column}: MAE={mae:.3f}, RMSE={rmse:.3f}, MAPE={mape:.2f}%")

In [ ]:
joblib.dump(model, 'autots_model.joblib')
print("Model saved as 'autots_model.joblib'")